# Consider LASSO problem

$$\min_{x,z}\frac12 ||Ax-b||_2^2 + \lambda ||z||_1$$
$$s.t. \enspace x-z=0$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def LASSO_obj(A,x,b,l,z):
    return np.linalg.norm(A.dot(x)-b)**2 + l*np.linalg.norm(z,1)

In [3]:
def LASSO_st(x,z):
    if x-z == 0:
        return True
    else:
        return False

Augmented Lagrangian：
$$L(x,z,v) = \min_{x,z}\frac12 ||Ax-b||_2^2 + \lambda ||z||_1 + v^T(x-z) + \frac{\rho}{2}||x-z||_2^2$$

In [4]:
def Aug_Lag(A,x,b,z,l,v,r):
    return LASSO_obj(A,x,b,l,z)+ v.T.dot(x-z) + r/2*np.linalg.norm(x-z)**2

To minimize only for x, the following partial differential equation must hold：
$$A^TAx-b+v+\rho x-\rho z = 0$$
$$x_{k+1} = \argmin_x L(x,z_{k},v_{k})$$

In [5]:
def argmin_x_L(A,b,v,r,z):
    return np.linalg.lstsq(A.T@A,A.T@b - v + r*z,rcond=None)[0]

For the nondifferentiable 1-norm, we consider archive this by simple gredient decent
$$z_{k+1} = \argmin_x L(x_{k+1},z,v_{k})$$

In [6]:
def L_zpart(z,v,x,r,l):
    return l*np.linalg.norm(z,ord = 1)+v.T.dot(x-z) + r/2*np.linalg.norm(x-z)**2

In [12]:
def argmin_z_L(x,z,v,l,r):
    z_t = z
    dz = (r*z_t-r*x -v +l*np.sign(z_t))
    m=0
    while m< 100:
        dz = (r*z_t-r*x -v +l*np.sign(z_t))
        t=1/r
        z_t = z_t - t*dz
        m+=1
    return z_t


$$v_{k+1} = v_{k} + \rho(x_{k+1}-z_{k+1})$$

In [8]:
def v_update(v,x,z,r):
    return v + r*(x-z)

In [9]:
def ADMM(A,x,b,l,z,v,r,n):
    x_t = x
    z_t = z
    v_t = v
    for i in range(n):
        x_t = argmin_x_L(A,b,v_t,r,z_t)
        print(x_t)
        z_t = argmin_z_L(x_t,z_t,v_t,l,r)
        print(z_t)
        v_t = v_update(v_t,x_t,z_t,r)
    return [LASSO_obj(A,x_t,b,l,z_t),Aug_Lag(A,x_t,b,z_t,l,v_t,r),x_t,z_t,v_t]

In [27]:
n =3
A = 10*np.random.rand(n,n)
x_0 = np.zeros((n,1))
z_0 = np.zeros((n,1))
b = np.ones((n,1))
v_0 = 0.1*np.ones((n,1))
r = 2
l = 2

In [28]:
ADMM(A,x_0,b,l,z_0,v_0,r,100)

[[-0.010426  ]
 [ 0.04949104]
 [ 0.10153843]]
[[-0.960426  ]
 [-0.90050896]
 [-0.84846157]]
[[ 0.00933401]
 [-0.01078551]
 [ 0.0988789 ]]
[[ 0.00933401]
 [-0.01078551]
 [ 0.0988789 ]]
[[-0.00061372]
 [ 0.01574572]
 [ 0.10271251]]
[[1.99938628]
 [0.01574572]
 [0.10271251]]
[[ 0.57199417]
 [-0.25053906]
 [-0.27449695]]
[[0.57199417]
 [1.74946094]
 [1.72550305]]
[[-0.22817519]
 [ 0.22674748]
 [ 0.2305664 ]]
[[-0.22817519]
 [ 0.22674748]
 [ 0.2305664 ]]
[[-0.09930718]
 [ 0.12498245]
 [ 0.15393196]]
[[-0.09930718]
 [ 0.12498245]
 [ 0.15393196]]
[[-0.06663262]
 [ 0.10618233]
 [ 0.13362749]]
[[-0.06663262]
 [ 0.10618233]
 [ 0.13362749]]
[[-0.05873683]
 [ 0.10203755]
 [ 0.12848574]]
[[-0.05873683]
 [ 0.10203755]
 [ 0.12848574]]
[[-0.05683337]
 [ 0.1010691 ]
 [ 0.12722699]]
[[-0.05683337]
 [ 0.1010691 ]
 [ 0.12722699]]
[[-0.05637475]
 [ 0.10083817]
 [ 0.12692219]]
[[-0.05637475]
 [ 0.10083817]
 [ 0.12692219]]
[[-0.05626427]
 [ 0.10078273]
 [ 0.12684864]]
[[-0.05626427]
 [ 0.10078273]
 [ 0.12684

[0.6480979371589859,
 array([[0.64809794]]),
 array([[-0.05622921],
        [ 0.10076516],
        [ 0.12682529]]),
 array([[-0.05622921],
        [ 0.10076516],
        [ 0.12682529]]),
 array([[-2.],
        [-2.],
        [-2.]])]

# Consider following optimization

$$\min \sum_{i=1}^{n}x_i\log x_i$$
$$s.t.\enspace Ax = b$$

In [ ]:
def q2_obj(x):
    return x.T.dot(np.log(x))

In [ ]:
def q2_st(A,x,b):
    if A@x == b:
        return True
    else:
        return False

Standard Newton：

$$\begin{bmatrix}
\nabla^2f(x) & A^T \\
A & O
\end{bmatrix} 
\begin{bmatrix}
\Delta x \\
w
\end{bmatrix} 
=
\begin{bmatrix}
-\nabla f(x) \\
O
\end{bmatrix} 
$$

In [ ]:
def newton_matrix(A,x):
    global p
    H = np.diagflat(x**(-1))
    H_1 = np.concatenate((H,A.T),axis=1)
    H_2 = np.concatenate((A,np.zeros((p,p))),axis=1)
    return np.concatenate((H_1,H_2),axis=0)

In [ ]:
def newton_b(x):
    global p
    g_f = np.log(x)+1
    return -np.concatenate((g_f,np.zeros((p,1))),axis = 0)

Infeasible start Newton：
$$\begin{bmatrix}
\nabla^2f(x) & A^T \\
A & O
\end{bmatrix} 
\begin{bmatrix}
\Delta x \\
w
\end{bmatrix} 
=-
\begin{bmatrix}
\nabla f(x) \\
Ax-b
\end{bmatrix} 
$$

In [ ]:
def infeasible_newton_b(A,x,b):
    global p
    g_f = np.log(x)+1
    return -np.concatenate((g_f,A@x-b),axis = 0)

In [ ]:
def standard_Newton(A,x_0,eps,alpha,beta):
    global n
    l = 1
    x = x_0
    m = 0
    while l/2 > eps:
        sol = np.linalg.lstsq(newton_matrix(A,x),newton_b(x),rcond=None)[0]
        sol_n = sol[:n]
        t = 1
        while q2_obj(x+t*sol_n) > q2_obj(x) + alpha*t*(np.log(x)+1).T.dot(sol_n):
            t = beta*t
        x = x+t*sol_n
        l = (1/x).T.dot(sol_n)*(t**2)
        m +=1
    return [x,q2_obj(x),m]

In [ ]:
def norm_r(x,v,A,b):
    return (np.linalg.norm((A.T@v + np.log(x)+1 ))**2 + np.linalg.norm((A@x - b))**2)**0.5

In [ ]:
def infeasible_start_Newton(A,x_0,v_0,b,eps,alpha,beta):
    global n
    l = 1
    x = x_0
    v = v_0
    m = 0
    while norm_r(x,v,A,b) > eps:
        sol = np.linalg.lstsq(newton_matrix(A,x),infeasible_newton_b(A,x,b),rcond=None)[0]
        dx = sol[:n]
        dv = sol[n:] - v
        t = 1
        while norm_r(x+t*dx,v+t*dv,A,b) > (1-alpha*t)*norm_r(x,v,A,b):
            t = beta*t
        x = x+t*dx
        v = v + t*dv
        m+=1
    return [x,q2_obj(x),m]

In [ ]:
n = 100
p=30
A = np.random.rand(p,n)
while np.linalg.matrix_rank(A)<p:
    A = np.random.rand(p,n)
x_0 = np.random.rand(n,1)
b = A@x_0

print(A,x_0,b)

[[0.26418398 0.79604118 0.79437835 ... 0.6034308  0.33506293 0.60481712]
 [0.4263807  0.29561196 0.25055046 ... 0.30865774 0.67173032 0.10781426]
 [0.84866001 0.20723868 0.78981388 ... 0.01503288 0.72939812 0.61396043]
 ...
 [0.11229131 0.87073789 0.96914675 ... 0.43488192 0.24317722 0.03630324]
 [0.51384799 0.74140843 0.04915594 ... 0.57829479 0.81370324 0.23148103]
 [0.56296755 0.36904464 0.01913013 ... 0.6524885  0.56110953 0.03536762]] [[0.91325976]
 [0.59596692]
 [0.47732267]
 [0.47595751]
 [0.38064756]
 [0.22698367]
 [0.09684551]
 [0.32782734]
 [0.11436357]
 [0.24151927]
 [0.32051032]
 [0.20979528]
 [0.38588719]
 [0.0437107 ]
 [0.07261632]
 [0.6778635 ]
 [0.31923509]
 [0.61336056]
 [0.09781155]
 [0.68234066]
 [0.97340201]
 [0.65681929]
 [0.58260615]
 [0.18439404]
 [0.0193759 ]
 [0.11371189]
 [0.9897346 ]
 [0.51826413]
 [0.87068548]
 [0.23467853]
 [0.48089072]
 [0.95455876]
 [0.99263351]
 [0.19664253]
 [0.27156292]
 [0.5241332 ]
 [0.12817739]
 [0.63794379]
 [0.01276613]
 [0.535256

In [ ]:
standard_Newton(A,x_0,1e-12,0.3,0.5)

[array([[0.55370011],
        [0.57320251],
        [0.39601599],
        [0.27948336],
        [0.38649772],
        [0.32871353],
        [0.48190679],
        [0.25309169],
        [0.53240959],
        [0.3778908 ],
        [0.5840181 ],
        [0.30738829],
        [0.50741592],
        [0.17635787],
        [0.4407153 ],
        [0.40023768],
        [0.28015118],
        [0.52075281],
        [0.24765085],
        [0.58514292],
        [0.55884283],
        [0.5230829 ],
        [0.39895493],
        [0.41125473],
        [0.18708439],
        [0.46358594],
        [0.33242708],
        [0.5064464 ],
        [0.57292513],
        [0.56949084],
        [0.31198298],
        [0.6756059 ],
        [0.62622882],
        [0.34715801],
        [0.52157338],
        [0.5486966 ],
        [0.44779635],
        [0.43412557],
        [0.27784128],
        [0.25925555],
        [0.27201858],
        [0.85664492],
        [0.5113722 ],
        [0.29252501],
        [0.32747354],
        [0

In [ ]:
v_0 = np.zeros((p,1))

In [ ]:
infeasible_start_Newton(A,x_0,v_0,b,1e-12,0.3,0.5)

[array([[0.55370011],
        [0.57320251],
        [0.39601599],
        [0.27948336],
        [0.38649772],
        [0.32871353],
        [0.48190679],
        [0.25309169],
        [0.53240959],
        [0.3778908 ],
        [0.5840181 ],
        [0.30738829],
        [0.50741592],
        [0.17635787],
        [0.4407153 ],
        [0.40023768],
        [0.28015118],
        [0.52075281],
        [0.24765085],
        [0.58514292],
        [0.55884283],
        [0.5230829 ],
        [0.39895493],
        [0.41125473],
        [0.18708439],
        [0.46358594],
        [0.33242708],
        [0.5064464 ],
        [0.57292513],
        [0.56949084],
        [0.31198298],
        [0.6756059 ],
        [0.62622882],
        [0.34715801],
        [0.52157338],
        [0.5486966 ],
        [0.44779635],
        [0.43412557],
        [0.27784128],
        [0.25925555],
        [0.27201858],
        [0.85664492],
        [0.5113722 ],
        [0.29252501],
        [0.32747354],
        [0

In [ ]:
x_1 = np.ones((n,1))

In [ ]:
infeasible_start_Newton(A,x_1,v_0,b,1e-12,0.3,0.5)

[array([[0.55370011],
        [0.57320251],
        [0.39601599],
        [0.27948336],
        [0.38649772],
        [0.32871353],
        [0.48190679],
        [0.25309169],
        [0.53240959],
        [0.3778908 ],
        [0.5840181 ],
        [0.30738829],
        [0.50741592],
        [0.17635787],
        [0.4407153 ],
        [0.40023768],
        [0.28015118],
        [0.52075281],
        [0.24765085],
        [0.58514292],
        [0.55884283],
        [0.5230829 ],
        [0.39895493],
        [0.41125473],
        [0.18708439],
        [0.46358594],
        [0.33242708],
        [0.5064464 ],
        [0.57292513],
        [0.56949084],
        [0.31198298],
        [0.6756059 ],
        [0.62622882],
        [0.34715801],
        [0.52157338],
        [0.5486966 ],
        [0.44779635],
        [0.43412557],
        [0.27784128],
        [0.25925555],
        [0.27201858],
        [0.85664492],
        [0.5113722 ],
        [0.29252501],
        [0.32747354],
        [0